# Import

In [1]:
import _pickle
import numpy as np
import pandas as pd
from mlxtend.evaluate import bias_variance_decomp
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              ExtraTreesClassifier, RandomForestClassifier,
                              StackingClassifier, VotingClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (SCORERS, accuracy_score, average_precision_score,
                             confusion_matrix, make_scorer, matthews_corrcoef,
                             precision_score, recall_score, roc_auc_score)
from sklearn.model_selection import (GridSearchCV, LeaveOneOut,
                                     RandomizedSearchCV, StratifiedKFold,
                                     cross_validate)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier

# GV and Func

In [2]:
def specificity(y_true, y_pred):
    return (2 * accuracy_score(y_true, y_pred)) - recall_score(y_true, y_pred)
    # tn, fp, fn, tp = confusion_matrix(y_true=y_true, y_pred=y_pred).ravel()
    # return tn / (tn + fp)


scoring = {
    "accuracy": make_scorer(accuracy_score),
    "mcc": make_scorer(matthews_corrcoef),
    "precision": make_scorer(precision_score),
    "specificity": make_scorer(specificity, greater_is_better=True),
    "sensitivity": make_scorer(recall_score),
    "auroc": make_scorer(roc_auc_score),
    "ap": make_scorer(average_precision_score),
}

In [3]:
clfs = {
    'DT':
    DecisionTreeClassifier(random_state=0),
    'LR':
    LogisticRegression(max_iter=10000, random_state=0),
    'RFC':
    RandomForestClassifier(random_state=0),
    'SVC(kernel = rbf)':
    SVC(kernel='rbf', random_state=0),
    'SVC(kernel = rbf, tuned)':
    SVC(kernel='rbf', C=5.44, gamma=.00237, random_state=0),
    'SVC(kernel = linear)':
    SVC(kernel='linear', random_state=0),
    'SVC(kernel = poly)':
    SVC(kernel='poly', random_state=0),
    'SVC(kernel = sigmoid)':
    SVC(kernel='sigmoid', random_state=0),
    'EXT':
    ExtraTreesClassifier(random_state=0),
    'GNB':
    GaussianNB(),
    "ADB":
    AdaBoostClassifier(random_state=0),
    "LDA":
    LinearDiscriminantAnalysis(),
    "DT":
    DecisionTreeClassifier(random_state=0),
    "KNN":
    KNeighborsClassifier(),
    'BG':
    BaggingClassifier(random_state=0),
    'BGsvmrbf':
    BaggingClassifier(base_estimator=SVC(kernel='rbf',
                                         random_state=0,
                                         probability=True),
                      random_state=0)
}

In [4]:
skf = StratifiedKFold(
    n_splits=10,
    random_state=0,
    shuffle=True)

In [5]:
def CV(X, y, model, cv=10):
    scores = cross_validate(model,
                            X,
                            y,
                            cv=cv,
                            scoring=scoring,
                            return_train_score=True,
                            return_estimator=True,
#                             verbose=1
                           )

    return {
        'ACC(%)': np.mean(scores["test_accuracy"]) * 100,
        'SN(%)': np.mean(scores["test_sensitivity"]) * 100,
        'SP(%)': np.mean(scores["test_specificity"]) * 100,
        'MCC': np.mean(scores["test_mcc"]),
    }

In [6]:
def IND(X, y, test_X, test_y, model):
    model.fit(X, y)
    pred_y = model.predict(test_X)

    return {
        'ACC(%)': accuracy_score(test_y, pred_y) * 100,
        'SE(%)': recall_score(test_y, pred_y) * 100,
        'SP(%)': specificity(test_y, pred_y) * 100,
        'MCC': matthews_corrcoef(test_y, pred_y),
    }

In [7]:
def JK(X, y, model, verbose = False):
    loo = LeaveOneOut()
    original = []
    pred = []
    i = 0
    for train_idx, test_idx in loo.split(X):
        print(i, end = '\t')
        i += 1
#         print(test_idx, end=',')
        model.fit(X[train_idx], y[train_idx])
        original.append(y[test_idx])
        pred.append(model.predict(X[test_idx]))
        
    return {
        'ACC(%)': accuracy_score(original, pred) * 100,
        'SE(%)': recall_score(original, pred) * 100,
        'SP(%)': specificity(original, pred) * 100,
        'MCC': matthews_corrcoef(original, pred),
    }

# Load data

In [8]:
file_name = '../Features/rf452.npz'

In [9]:
data = np.load(file_name)

X = data['X']
y = data['y']
test_X = data['test_X']
test_y = data['test_y']

print(X.shape, test_X.shape)

(1424, 452) (356, 452)


In [10]:
sc = StandardScaler()
X = sc.fit_transform(X)
test_X = sc.transform(test_X)

In [11]:
np.max(test_X), np.max(X), np.min(test_X), np.min(X)

(13.04237827911235, 15.95060459925624, -7.071253875525869, -9.861130594899153)

# Test

In [12]:
report_cv = []
report_ind = []

for name, clf in clfs.items():
    print(name)
    report_cv.append(CV(X, y, clf, cv= skf))
    report_cv[-1]['Classifier'] = name
    
    report_ind.append(IND(X, y, test_X, test_y, clf))
    report_ind[-1]['Classifier'] = name

DT
LR
RFC
SVC(kernel = rbf)
SVC(kernel = rbf, tuned)
SVC(kernel = linear)
SVC(kernel = poly)
SVC(kernel = sigmoid)
EXT
GNB
ADB
LDA
KNN
BG
BGsvmrbf


In [13]:
report_jk = []
name = 'SVC(kernel = rbf, tuned)'
report_jk.append(JK(X, y, clfs[name]))
report_jk[-1]['Classifier'] = name

print(clfs[name])

report_cv = pd.DataFrame(report_cv)
report_ind = pd.DataFrame(report_ind)
report_jk = pd.DataFrame(report_jk)

print(report_cv)

print(report_ind)

print(report_jk)

0	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	76	77	78	79	80	81	82	83	84	85	86	87	88	89	90	91	92	93	94	95	96	97	98	99	100	101	102	103	104	105	106	107	108	109	110	111	112	113	114	115	116	117	118	119	120	121	122	123	124	125	126	127	128	129	130	131	132	133	134	135	136	137	138	139	140	141	142	143	144	145	146	147	148	149	150	151	152	153	154	155	156	157	158	159	160	161	162	163	164	165	166	167	168	169	170	171	172	173	174	175	176	177	178	179	180	181	182	183	184	185	186	187	188	189	190	191	192	193	194	195	196	197	198	199	200	201	202	203	204	205	206	207	208	209	210	211	212	213	214	215	216	217	218	219	220	221	222	223	224	225	226	227	228	229	230	231	232	233	234	235	236	237	238	239	240	241	242	243	244	245	246	247	248	249	250	251	252	253	254	255	256	257	258	259	260	261	262	263	264	265	266	267	268	269	270	271	272	273	274	275	276	27

In [14]:
# ranked_features.to_csv('Feature_ranking_' + file_name.split('.')[0] + '.csv', index = False)
report_cv.to_csv('Report_CV_' + file_name.split('.')[0] + '.csv', index = False)
report_ind.to_csv('Report_IND_' + file_name.split('.')[0] + '.csv', index=False)
report_jk.to_csv('Report_JK_' + file_name.split('.')[0] + '.csv', index=False)

# Tuning

In [15]:
# parma_dist = dict(
#     C = np.logspace(-2, 4, 10),
#     gamma = np.logspace(-9, 3, 10)
# )

# rnd = RandomizedSearchCV(clf,
#                          param_distributions=parma_dist,
#                          scoring='accuracy',
#                          cv=skf,
#                          random_state=0,
#                          n_jobs=4,
#                          verbose=1)

# rnd.fit(X, y)

In [16]:
parma_dist = dict(
    C = np.logspace(-2, 4, 10),
    gamma = np.logspace(-9, 3, 10)
)

grid = GridSearchCV(clfs['SVC(kernel = rbf)'],
                    param_grid=parma_dist,
                    cv=skf,
                    n_jobs=-1,
                    verbose=1,
                    scoring='accuracy')

grid.fit(X, y)

print(grid.best_params_, grid.best_score_, IND(X, y, test_X, test_y, grid.best_estimator_))
print(grid.param_grid)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   26.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  7.4min finished


{'C': 4.6415888336127775, 'gamma': 0.004641588833612773} 0.9136462129419876 {'ACC(%)': 91.29213483146067, 'SE(%)': 88.20224719101124, 'SP(%)': 94.3820224719101, 'MCC': 0.827424158774056}
{'C': array([1.00000000e-02, 4.64158883e-02, 2.15443469e-01, 1.00000000e+00,
       4.64158883e+00, 2.15443469e+01, 1.00000000e+02, 4.64158883e+02,
       2.15443469e+03, 1.00000000e+04]), 'gamma': array([1.00000000e-09, 2.15443469e-08, 4.64158883e-07, 1.00000000e-05,
       2.15443469e-04, 4.64158883e-03, 1.00000000e-01, 2.15443469e+00,
       4.64158883e+01, 1.00000000e+03])}


In [17]:
parma_dist = dict(
    C = np.linspace(1, 21, 10),
    gamma = np.linspace(0.0002, 0.01, 10)
)

grid = GridSearchCV(clfs['SVC(kernel = rbf)'],
                    param_grid=parma_dist,
                    cv=skf,
                    n_jobs=-1,
                    verbose=1,
                    scoring='accuracy')

grid.fit(X, y)

print(grid.best_params_, grid.best_score_, IND(X, y, test_X, test_y, grid.best_estimator_))
print(grid.param_grid)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  6.4min finished


{'C': 5.444444444444445, 'gamma': 0.0023777777777777777} 0.9199694671525656 {'ACC(%)': 92.13483146067416, 'SE(%)': 89.32584269662921, 'SP(%)': 94.9438202247191, 'MCC': 0.8440296311368944}
{'C': array([ 1.        ,  3.22222222,  5.44444444,  7.66666667,  9.88888889,
       12.11111111, 14.33333333, 16.55555556, 18.77777778, 21.        ]), 'gamma': array([0.0002    , 0.00128889, 0.00237778, 0.00346667, 0.00455556,
       0.00564444, 0.00673333, 0.00782222, 0.00891111, 0.01      ])}


# Ensemble

In [18]:
def JKSL(X, y, verbose=False):
    original = []
    pred = []

    loo = LeaveOneOut()
    model = get_super_learner(X, folds=10)

    dummy = np.zeros((len(X[0]), 9)).T

    i = 0
    for train_idx, test_idx in loo.split(X):
        print(i, end='\t')
        i += 1
        model.fit(X[train_idx], y[train_idx])

        original.append(y[test_idx])

        test_data = np.vstack([dummy, X[test_idx]])
        #         print(test_data.shape)
        pred.append(model.predict(test_data)[-1])

    return {
        'ACC(%)': accuracy_score(original, pred) * 100,
        'SE(%)': recall_score(original, pred) * 100,
        'SP(%)': specificity(original, pred) * 100,
        'MCC': matthews_corrcoef(original, pred),
    }


In [20]:
# make a prediction with a stacking ensemble
# define the base models
level0 = list()

level0.append((
    'VC',
    VotingClassifier(
        [
            ('DT', DecisionTreeClassifier(random_state=0)),
            ('ABC', AdaBoostClassifier(random_state=0)),
            ('LDA', LinearDiscriminantAnalysis()),
            #             ('SVCL', SVC(kernel='poly', random_state=0, probability=True))
        ],
        voting='soft',
        n_jobs=-1,
        # weights=[1, 1, 1.5]
    )))
# level0.append(('ext', ext(random_state=0)))
level0.append(('svm(rbf, tuned)',
               SVC(kernel='rbf',
                   C=5.44,
                   gamma=.00237,
                   random_state=0,
                   probability=True)))
# level0.append(('svc(poly)', SVC(kernel='poly',
#                                 random_state=0,
#                                 probability=True)))

# define meta learner model
level1 = LogisticRegression(solver='liblinear' ,random_state=0)
# level1 = SVC(kernel='linear', random_state=0)
# level1 = LinearDiscriminantAnalysis()
# define the stacking ensemble
model = StackingClassifier(estimators=level0,
                           final_estimator=level1,
                           stack_method='predict_proba',
                           cv=skf)
# fit the model on all available data
model.fit(X, y)
# make a prediction for one example
yhat = model.predict(test_X)
print('Predicted Accuracy: %f' % accuracy_score(test_y, yhat))
print('Predicted Sensitivity: %f' % recall_score(test_y, yhat))
print('Predicted Specificity: %f' % specificity(test_y, yhat))
print('Predicted MCC: %f' % matthews_corrcoef(test_y, yhat))

Predicted Accuracy: 0.926966
Predicted Sensitivity: 0.898876
Predicted Specificity: 0.955056
Predicted MCC: 0.855283


In [21]:
print('10-Fold CV')
print(CV(X, y, model, StratifiedKFold(n_splits=10, shuffle=True, random_state=0)))

print(model)

print('JK')
print(JKSL(X, y, model))

10-Fold CV
{'ACC(%)': 92.06786171574905, 'SN(%)': 92.4197965571205, 'SP(%)': 91.71592687437759, 'MCC': 0.8427047226262575}
StackingClassifier(cv=StratifiedKFold(n_splits=10, random_state=0, shuffle=True),
                   estimators=[('VC',
                                VotingClassifier(estimators=[('DT',
                                                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                     class_weight=None,
                                                                                     criterion='gini',
                                                                                     max_depth=None,
                                                                                     max_features=None,
                                                                                     max_leaf_nodes=None,
                                                                       

# Others

In [22]:
vt = VotingClassifier(estimators=[
    ('gnb', GaussianNB()),
    ('adb', AdaBoostClassifier(random_state=0)),
    ('svmrbf', SVC(C=5.44, gamma=0.00273, probability=True, random_state=0)),
    ('ext', ExtraTreesClassifier(random_state=0))
],
                      voting='hard',
                      weights=[0.3, 0.3, 0.7, 0.3])

vt.fit(X, y)

p = vt.predict(test_X)

print(accuracy_score(test_y, p))

0.901685393258427


In [ ]:
loss, bias, var = bias_variance_decomp(model, X, y, test_X, test_y, loss='0-1_loss', random_seed=0)
print('0-1 loss: ', loss)
print('Bias : ', bias)
print('Variance : ', var)

In [26]:
model = clfs['SVC(kernel = rbf, tuned)']
mse, bias, var = bias_variance_decomp(model,
                                         X,
                                         y,
                                         test_X,
                                         test_y,
                                         loss='mse',
                                         random_seed=0)
print('Mean Square Error: ', mse)
print('Bias : ', bias)
print('Variance : ', var)

Mean Square Error:  0.08980337078651686
Bias :  0.07276573033707866
Variance :  0.017037640449438205
